In [1]:
import os
import pandas as pd
from datasets import concatenate_datasets, Dataset
import numpy as np
from sentence_transformers import SentenceTransformer
import torch
from langchain.text_splitter import RecursiveCharacterTextSplitter
import faiss
import pickle
import multiprocessing as mp
from transformers import pipeline

c:\Users\User\anaconda3\envs\mypaper\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain.vectorstores import FAISS
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain import LLMChain


In [3]:
def read_txt_files_to_dataframe(directory):
    data = []
    
 
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r', encoding='utf-8') as file:
                content = file.read()
                title = os.path.splitext(filename)[0]
                data.append({'Title': title, 'Content': content})
    
    df = pd.DataFrame(data)
    
    return df

資料整理

In [4]:
directory_path_1 = r'C:\Users\User\Desktop\code\word_embedding_law\行政-財政及金融.txt'
directory_path_2 = r'C:\Users\User\Desktop\code\word_embedding_law\行政-經濟及能源.txt'

commercial_df = read_txt_files_to_dataframe(directory_path_1)
eco_df = read_txt_files_to_dataframe(directory_path_2)

In [5]:
commercial_dataset = Dataset.from_pandas(commercial_df)
eco_dataset = Dataset.from_pandas(eco_df)

In [6]:
def concatenate_text(examples):
    return {
        "text": examples["Title"]
        + " \n "
        + examples["Content"]

    }


commercial_dataset = commercial_dataset.map(concatenate_text)
eco_dataset = eco_dataset.map(concatenate_text)

Map: 100%|██████████| 96/96 [00:00<00:00, 8595.80 examples/s]


In [7]:
law_dataset = concatenate_datasets([eco_dataset, commercial_dataset])

In [8]:
law_text = ' '.join(law_dataset['text'])

In [9]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n"], chunk_size=200, chunk_overlap=50)
text = text_splitter.split_text(law_text)

模型

In [10]:
model_ckpt = "intfloat/multilingual-e5-large"

model = SentenceTransformer(model_ckpt)

In [11]:
device = torch.device("cuda")

embedding 沒事別按

In [40]:
embeddings = model.encode(text, device='cuda')

In [41]:
with open('law_Sentence_embeddings', 'wb') as f:
    pickle.dump(embeddings, f)



打開儲存的embedding

In [12]:
with open('law_Sentence_embeddings', 'rb') as f:
    embeddings = pickle.load(f)

In [213]:
faiss.normalize_L2(embeddings)
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(embeddings)

In [214]:
faiss.write_index(index, "C:/Users/User/Desktop/code/word_embedding_law/faiss_index.bin")

In [215]:
index = faiss.read_index("C:/Users/User/Desktop/code/word_embedding_law/faiss_index.bin")

RAG部分

In [216]:
openai_api_key = "sk-proj-eyJDSUhWSIYkewbENUGuT3BlbkFJyIDoMtck7604rfRajOV7"

In [301]:
def generate_sub_queries(query, num_sub_queries=3):


    # 設定 PromptTemplate
    prompt = PromptTemplate(
        input_variables=["query", "num_sub_queries"],
        template="請使用繁體中文生成出{num_sub_queries}個與{query}相關的問題，以增加檢索的範圍"
    )

    # 初始化 OpenAI 的 LLM
    llm = OpenAI(temperature=0.7, openai_api_key=openai_api_key)

    # 創建 LLMChain
    llm_chain = LLMChain(llm=llm, prompt=prompt)

    # 生成 sub-queries
    sub_queries_text = llm_chain.run(query=query, num_sub_queries=num_sub_queries).strip()
    sub_queries = sub_queries_text.split('\n')  # 假設 LLM 返回每行一個 sub-query

    return sub_queries

In [302]:
query = "哪些人要繳菸酒稅"

#sub_queries = generate_sub_queries(query)

In [303]:
def Retrieval_Documents(query):
    query_embedding = model.encode([query])
    faiss.normalize_L2(query_embedding)
    D, I = index.search(query_embedding, k=5)
    retrieved_docs = [text[i] for i in I[0]]
    custom_context = "\n\n".join(retrieved_docs)

    return  retrieved_docs, custom_context

In [304]:
Retrieval_Documents(query)[0]

['* 2 菸酒類稅為國家稅，由財政部稅務署所屬稅務機關徵收之。\n\n* 3 菸酒類稅稅率規定如左。（一）菸類稅分菸葉菸絲兩種，菸葉稅按照產區核定完稅價格徵收百分之三十，菸絲稅徵收百分之十五。（二）酒類稅按產地核定完稅價格徵收百分之四十。前項第一款刨絲之菸葉，仍應先納菸葉稅。',
 '* 4 菸酒稅之納稅義務人如下：一、國內產製之菸酒，為產製廠商。二、委託代製之菸酒，為受託之產製廠商。三、國外進口之菸酒，為收貨人、提貨單或貨物持有人。四、法院及其他機關拍賣尚未完稅之菸酒，為拍定人。五、免稅菸酒因轉讓或移作他用而不符免稅規定者，為轉讓或移作他用之人或貨物持有人。前項第二款委託代製之菸酒，委託廠商為產製應稅菸酒之廠商者，得向主管稽徵機關申請以委託廠商為納稅義務人。',
 '> 釋：查產製廠商未依規定申報菸品健康福利捐，應加徵滯報金、怠報金，財政部於「菸酒稅稽徵規則」第十三條第二項已規定，菸品產製廠商於申報菸稅時，應同時向主管稽徵機關申報繳納菸品健康福利捐，並明定菸品健康福利捐之徵收準用本法有關菸酒稅之規定，因涉及人民權利義務，配合行政程序法之施行，爰增訂相關規定。',
 '> 釋：明定菸酒稅之納稅義務人。\n\n* 5 菸酒有下列情形之一者，免徵菸酒稅：一、用作產製另一應稅菸酒者。二、運銷國外者。三、參加展覽，於展覽完畢原件復運回廠或出口者。四、旅客自國外隨身攜帶之自用菸酒或調岸船員攜帶自用菸酒，未超過政府規定之限量者。',
 '* 3 菸酒稅於菸酒出廠或進口時徵收之。菸酒有下列情形之一，視為出廠：一、在廠內供消費者。二、在廠內加工為非應稅產品者。三、在廠內因依法強制執行或其他原因而移轉他人持有者。四、產製廠商申請註銷登記時之庫存菸酒。五、未稅移運至加工、包裝場所或存儲未稅倉庫及廠內，有遇火焚毀或落水沉沒及其他人力不可抵抗災害以外之情事，致短少者。']

In [305]:
def RAG_LLM_chain(query):    
    prompt = PromptTemplate(
        input_variables=["context","query"],
        template="""
        
你現在是一個法律顧問機器人，請根據以下要求回答問題：
一、使用者會輸入{query}，你必須根據{context}回答
二、如{context}中有條文修正或是刪除的補充，忽略它
三、請以繁體中文回答

"""
    )

    # 使用 LLMChain 和 OpenAI LLM
    llm = OpenAI(temperature=0.7, openai_api_key=openai_api_key, max_tokens=700)

    llm_chain = LLMChain(llm=llm, prompt=prompt)

    # 傳遞自定義的上下文到 LLM
    final_result = llm_chain.run(context=Retrieval_Documents(query)[1], query=query)

    print("Final Result:\n", final_result)

生成回應

In [236]:
print("Our query:", query)
#print("Generated Sub-queries:", sub_queries)
print("="* 50)

RAG_LLM_chain(query)

for i, doc in enumerate(Retrieval_Documents(query)[0]):
    print("="* 50)
    print(f"Document {i+1}:")
    print(Retrieval_Documents(query)[0][i])
    print("-" * 50)

Our query: 遺產超過多少錢時要繳遺產稅
Final Result:
 二、根據* 29 遺產稅申報義務人違反第十七條之規定不依限申報者，處一千元以下之罰鍰，並通知補報。

* 30 納稅義務人意圖減免稅額而為虛偽申報或有隱匿遺產之行為者，除照補稅額外並處以所隱稅額一倍至三倍之罰鍰，其觸犯刑法者，應依刑法處斷。



* 15 遺產淨值超過二萬元者，起徵遺產稅，依左列稅率按級計算課徵之：一、超過二萬元至四萬元者，就其超過額課徵百分之四。二、超過四萬元至六萬元者，就其超過額課徵百分之五。三、超過六萬元至八萬元者，就其超過額課徵百分之六。四、超過八萬元至十萬元者，就其超過額課徵百分之七。五、超過十萬元至十二萬元者，就其超過額課徵百分之九。六、超過十二萬元至十四萬元者，就其超過額課徵百分之十一。七、超過十四萬元至十六萬元者，就其超過額課徵百分之十三。八、超過十六萬元至十八萬元者，就其超過額課徵百分之十五。九、超過十八萬元至二十萬元者，就其超過額課徵百分之十七。十、超過二十萬元至二十五萬元者，就其超過額課徵百分之二十。十一、超過二十五萬元至三十萬元者，就其超過額課徵百分之二十三。十二、超過三十萬元至三十五萬元者，就其超過額課徵百分之二十六。十三、超過三十五萬元至四十萬元者，就其超過額課徵
Document 1:
* 29 遺產稅申報義務人違反第十七條之規定不依限申報者，處一千元以下之罰鍰，並通知補報。

* 30 納稅義務人意圖減免稅額而為虛偽申報或有隱匿遺產之行為者，除照補稅額外並處以所隱稅額一倍至三倍之罰鍰，其觸犯刑法者，應依刑法處斷。
--------------------------------------------------
Document 2:


* 15 遺產淨值超過二萬元者，起徵遺產稅，依左列稅率按級計算課徵之：一、超過二萬元至四萬元者，就其超過額課徵百分之四。二、超過四萬元至六萬元者，就其超過額課徵百分之五。三、超過六萬元至八萬元者，就其超過額課徵百分之六。四、超過八萬元至十萬元者，就其超過額課徵百分之七。五、超過十萬元至十二萬元者，就其超過額課徵百分之九。六、超過十二萬元至十四萬元者，就其超過額課徵百分之十一。七、超過十四萬元至十六萬元者，就其超過額課徵百分之十三。八、超過十六萬元至十八萬元者，就其超過額課徵百分之十五。九、超過十八萬元

嘗試做做看聚類

AHC

In [276]:
import umap
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import fcluster
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModel, BertConfig, GPTNeoForCausalLM, GPT2Tokenizer
import gc
from scipy.cluster.hierarchy import linkage, dendrogram, to_tree
from scipy.spatial.distance import cosine, euclidean
import heapq

In [309]:
umap_model = umap.UMAP(n_components=3, random_state=42)
reduced_embeddings = umap_model.fit_transform(embeddings)

c:\Users\User\anaconda3\envs\mypaper\lib\site-packages\umap\umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


In [311]:
Z = linkage(reduced_embeddings, method='weighted')

In [312]:
root = to_tree(Z)

In [313]:
class Node:
    def __init__(self, id, left=None, right=None, is_leaf=False, sentence=None, embedding=None):
        self.id = id
        self.left = left
        self.right = right
        self.is_leaf = is_leaf
        self.sentence = sentence
        self.embedding = embedding

In [315]:
def build_tree(node, sentences, embeddings):
    if node.is_leaf():
        return Node(node.id, is_leaf=True, sentence=sentences[node.id], embedding=embeddings[node.id])
    left = build_tree(node.left, sentences, embeddings)
    right = build_tree(node.right, sentences, embeddings)
    return Node(node.id, left=left, right=right)


tree = build_tree(root, text, reduced_embeddings)

In [316]:
def query_search(node, query_embedding, top_k):
    def search(node, query_embedding, results):
        if node.is_leaf:
            distance = cosine(query_embedding, node.embedding)
            heapq.heappush(results, (-distance, node.sentence))
            if len(results) > top_k:
                heapq.heappop(results)
        else:
            search(node.left, query_embedding, results)
            search(node.right, query_embedding, results)
    
    results = []
    search(node, query_embedding, results)
    
    return [(sentence, -distance) for distance, sentence in sorted(results, reverse=True)]

In [320]:
top_k = 5
query = "哪些人要繳菸酒稅"
query_embedding = umap_model.transform(model.encode([query]))[0]


In [321]:
def RAG_cluster_LLM(query, results):    
    prompt = PromptTemplate(
        input_variables=["context","query"],
        template="""
        
你現在是一個法律顧問機器人，請根據以下要求回答問題：
一、使用者會輸入{query}，你必須根據{context}回答
二、如{context}中有條文修正或是刪除的補充，忽略它
三、請以繁體中文回答

"""
    )

    # 使用 LLMChain 和 OpenAI LLM
    llm = OpenAI(temperature=0.7, openai_api_key=openai_api_key, max_tokens=900)

    llm_chain = LLMChain(llm=llm, prompt=prompt)

    # 傳遞自定義的上下文到 LLM
    final_result = llm_chain.run(context=results, query=query)

    print("Final Result:\n", final_result)

In [322]:
results = query_search(tree, query_embedding, top_k)

print(f"查詢: '{query}'")
print(f"前 {top_k} 個最相似的句子:")
for i, (sentence, similarity) in enumerate(results, 1):
    print(f"{i}. 文本: '{sentence}', 相似度: {1 - similarity:.4f}")

查詢: '哪些人要繳菸酒稅'
前 5 個最相似的句子:
1. 文本: '* 2 菸酒類稅為國家稅，由財政部稅務署所屬稅務機關徵收之。

* 3 菸酒類稅稅率規定如左。（一）菸類稅分菸葉菸絲兩種，菸葉稅按照產區核定完稅價格徵收百分之三十，菸絲稅徵收百分之十五。（二）酒類稅按產地核定完稅價格徵收百分之四十。前項第一款刨絲之菸葉，仍應先納菸葉稅。', 相似度: 1.0000
2. 文本: '* 12 產製廠商當月份出廠菸酒之應納稅款，應於次月十五日以前自行向公庫繳納，並依照財政部規定之格式填具計算稅額申報書，檢同繳款書收據向主管稽徵機關申報。無應納稅額者，仍應向主管稽徵機關申報。進口應稅菸酒，納稅義務人應向海關申報，並由海關於徵收關稅時代徵之。法院及其他機關拍賣尚未完稅之菸酒，拍定人應於提領前向所在地主管稽徵機關申報納稅。', 相似度: 0.9999
3. 文本: '* 4 菸酒稅之納稅義務人如下：一、國內產製之菸酒，為產製廠商。二、委託代製之菸酒，為受託之產製廠商。三、國外進口之菸酒，為收貨人、提貨單或貨物持有人。四、法院及其他機關拍賣尚未完稅之菸酒，為拍定人。五、免稅菸酒因轉讓或移作他用而不符免稅規定者，為轉讓或移作他用之人或貨物持有人。前項第二款委託代製之菸酒，委託廠商為產製應稅菸酒之廠商者，得向主管稽徵機關申請以委託廠商為納稅義務人。', 相似度: 0.9999
4. 文本: '> 釋：明定菸酒稅之納稅義務人。

* 5 菸酒有下列情形之一者，免徵菸酒稅：一、用作產製另一應稅菸酒者。二、運銷國外者。三、參加展覽，於展覽完畢原件復運回廠或出口者。四、旅客自國外隨身攜帶之自用菸酒或調岸船員攜帶自用菸酒，未超過政府規定之限量者。', 相似度: 0.9999
5. 文本: '* 3 菸酒稅於菸酒出廠或進口時徵收之。菸酒有下列情形之一，視為出廠：一、在廠內供消費者。二、在廠內加工為非應稅產品者。三、在廠內因依法強制執行或其他原因而移轉他人持有者。四、產製廠商申請註銷登記時之庫存菸酒。五、未稅移運至加工、包裝場所或存儲未稅倉庫及廠內，有遇火焚毀或落水沉沒及其他人力不可抵抗災害以外之情事，致短少者。', 相似度: 0.9998


In [323]:
results = [x[0] for x in results]

In [324]:
print("Our query:", query)
#print("Generated Sub-queries:", sub_queries)
print("="* 50)

RAG_cluster_LLM(query, results)

for i, doc in enumerate(results):
    print("="* 50)
    print(f"Document {i+1}:")
    print(results[i])
    

Our query: 哪些人要繳菸酒稅
Final Result:
 菸酒稅的納稅義務人有產製廠商、委託代製廠商、國外進口菸酒的收貨人或提貨單持有人、法院及其他機關拍賣尚未完稅的菸酒的拍定人、以及免稅菸酒轉讓或移作他用的人或貨物持有人。若委託廠商為產製應稅菸酒的廠商，則可向主管稽徵機關申請以委託廠商為納稅義務人。另外，菸酒稅有特定的情形免徵，包括用作產製另一應稅菸酒、運銷國外、參加展覽並於展覽完畢原件復運回廠或出口、旅客自國外隨身攜帶之自用菸酒或調岸船員攜帶自用菸酒，且未超過政府規定的限量。另外，菸酒稅於菸酒出廠或進口時徵收，而菸酒的出廠情形包括在廠內供消費者、在廠內加工為非應稅產品、在廠內因依法強制執行或其他原因而移轉他人持有、產製廠商申請註銷登記時之庫存菸酒、以及未稅移運至加工、包裝場所或存儲未稅倉庫及廠內，有遇火焚毀或落水沉沒及其他人力不可抵抗災害以外之情事，致短少者。
Document 1:
* 2 菸酒類稅為國家稅，由財政部稅務署所屬稅務機關徵收之。

* 3 菸酒類稅稅率規定如左。（一）菸類稅分菸葉菸絲兩種，菸葉稅按照產區核定完稅價格徵收百分之三十，菸絲稅徵收百分之十五。（二）酒類稅按產地核定完稅價格徵收百分之四十。前項第一款刨絲之菸葉，仍應先納菸葉稅。
Document 2:
* 12 產製廠商當月份出廠菸酒之應納稅款，應於次月十五日以前自行向公庫繳納，並依照財政部規定之格式填具計算稅額申報書，檢同繳款書收據向主管稽徵機關申報。無應納稅額者，仍應向主管稽徵機關申報。進口應稅菸酒，納稅義務人應向海關申報，並由海關於徵收關稅時代徵之。法院及其他機關拍賣尚未完稅之菸酒，拍定人應於提領前向所在地主管稽徵機關申報納稅。
Document 3:
* 4 菸酒稅之納稅義務人如下：一、國內產製之菸酒，為產製廠商。二、委託代製之菸酒，為受託之產製廠商。三、國外進口之菸酒，為收貨人、提貨單或貨物持有人。四、法院及其他機關拍賣尚未完稅之菸酒，為拍定人。五、免稅菸酒因轉讓或移作他用而不符免稅規定者，為轉讓或移作他用之人或貨物持有人。前項第二款委託代製之菸酒，委託廠商為產製應稅菸酒之廠商者，得向主管稽徵機關申請以委託廠商為納稅義務人。
Document 4:
> 釋：明定菸酒稅之納稅義務人。

* 5 菸酒有下列情形之一者，免徵菸酒稅：一、用作產製另一應稅菸酒者。二、運銷國外者